In [1]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
import numpy as np
import time
from datetime import datetime

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import  StandardScaler
from datetime import datetime
from sklearn.cluster import DBSCAN, OPTICS
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_pickle('Coingecko_getcoin_byid_res.pkl')
df.columns = range(df.columns.size)
df = df.T
df.head()

,id,symbol,name,asset_platform_id,block_time_in_minutes,hashing_algorithm,categories,description,country_origin,genesis_date,...,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,market_data,community_data,developer_data,public_interest_stats
0,01coin,zoc,01coin,None,0,NeoScrypt,[Masternodes],"{'en': '', 'de': '', 'es': '', 'fr': '', 'it':...",,None,...,2437,14.63,44.681,14.816,1.0,0.0,"{'current_price': {'aed': 0.00241217, 'ars': 0...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 11, 'stars': 15, 'subscribers': 7, '...","{'alexa_rank': 4678489, 'bing_matches': None}"
1,01coin,zoc,01coin,None,0,NeoScrypt,[Masternodes],"{'en': '', 'de': '', 'es': '', 'fr': '', 'it':...",,None,...,2437,14.63,44.681,14.816,1.0,0.0,"{'current_price': {'aed': 0.00241217, 'ars': 0...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 11, 'stars': 15, 'subscribers': 7, '...","{'alexa_rank': 4678489, 'bing_matches': None}"
2,0-5x-long-algorand-token,algohalf,0.5X Long Algorand Token,ethereum,0,None,[],{'en': '0.5X Long Algorand Token (ALGOHALF) is...,,None,...,11513,0.294,0.0,0.0,1.0,1.87,"{'current_price': {'aed': 59847, 'ars': 169857...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 0, 'stars': 0, 'subscribers': 0, 'to...","{'alexa_rank': 5253, 'bing_matches': None}"
3,0-5x-long-altcoin-index-token,althalf,0.5X Long Altcoin Index Token,ethereum,0,None,[],{'en': '0.5X Long Altcoin Index Token (ALTHALF...,,None,...,11447,0.294,0.0,0.0,1.0,1.87,"{'current_price': {'aed': 67986, 'ars': 192958...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 0, 'stars': 0, 'subscribers': 0, 'to...","{'alexa_rank': 5253, 'bing_matches': None}"
4,0-5x-long-balancer-token,balhalf,0.5X Long Balancer Token,ethereum,0,None,[],{'en': '0.5X Long Balancer Token (BALHALF) is ...,,None,...,12265,0.094,0.0,0.0,0.0,1.87,"{'current_price': {'aed': 38080, 'ars': 108040...","{'facebook_likes': None, 'twitter_followers': ...","{'forks': 0, 'stars': 0, 'subscribers': 0, 'to...","{'alexa_rank': 5253, 'bing_matches': None}"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12703 entries, 0 to 12702
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   id                               12703 non-null  object
 1   symbol                           12703 non-null  object
 2   name                             12703 non-null  object
 3   asset_platform_id                11237 non-null  object
 4   block_time_in_minutes            12703 non-null  object
 5   hashing_algorithm                1132 non-null   object
 6   categories                       12703 non-null  object
 7   description                      12703 non-null  object
 8   country_origin                   12678 non-null  object
 9   genesis_date                     815 non-null    object
 10  sentiment_votes_up_percentage    6294 non-null   object
 11  sentiment_votes_down_percentage  6294 non-null   object
 12  market_cap_rank                 

In [4]:
df.genesis_date = pd.to_datetime(df['genesis_date'], format='%Y-%m-%d', errors='coerce')

In [5]:
pd.DatetimeIndex(df['genesis_date']).to_julian_date().unique()

Float64Index([      nan, 2457976.5, 2457326.5, 2458087.5, 2457496.5, 2457704.5,
              2459127.5, 2457956.5, 2456901.5, 2457999.5,
              ...
              2457689.5, 2457659.5, 2458189.5, 2456716.5, 2458393.5, 2458391.5,
              2457803.5, 2456507.5, 2458514.5, 2458255.5],
             dtype='float64', name='genesis_date', length=588)

In [6]:
column_names = ['mcap_to_tvl_ratio', 'fdv_to_tvl_ratio', 'market_cap', 
                'price_change_percentage_60d_in_currency','total_supply',
                'ath_change_percentage', 'atl_change_percentage']

class getMarketPrice(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        X_out = pd.DataFrame(columns = column_names, index = X.index)
        
        for rowi in X.index:
            
            tempdf = pd.DataFrame.from_dict(X.loc[rowi])
            X_out.loc[rowi,:] = tempdf.loc['usd', column_names]
            
        return X_out   

In [7]:
X = df.copy(deep = True)
marketTrans = getMarketPrice()  
X_out_temp = marketTrans.fit_transform(X.loc[:,'market_data'])

In [8]:
X = pd.merge(X, X_out_temp, left_index=True, right_index=True)

In [9]:
X.market_cap_rank.fillna(value=-1, inplace=True)

X.sentiment_votes_down_percentage.fillna(value=0, inplace=True)

X.sentiment_votes_up_percentage.fillna(value=0, inplace=True)

X.coingecko_rank.fillna(value=-1, inplace=True)

X.asset_platform_id.fillna(value='Unknown', inplace=True)

X.hashing_algorithm.fillna(value='Other', inplace=True)

X.country_origin.fillna(value='Not_listed', inplace=True)

X.genesis_date.fillna(value='Not_available', inplace=True)

X.country_origin.replace('', 'Not_listed',inplace=True) # fix where value is ''

X.genesis_date = pd.to_datetime(X['genesis_date'], format='%Y-%m-%d', errors='coerce')

X['Jdate'] = pd.DatetimeIndex(X['genesis_date']).to_julian_date()

X['Jdate'].fillna(value=-100, inplace=True)

In [10]:
X['market_cap'].fillna(value=-100, inplace=True)
X['mcap_to_tvl_ratio'].fillna(value=-100, inplace=True)
X['mcap_to_tvl_ratio'].replace('-', -100, inplace=True)
X['fdv_to_tvl_ratio'].fillna(value=-100, inplace=True)
X['fdv_to_tvl_ratio'].replace(['-','?'], -100, inplace=True)
X['ath_change_percentage'].fillna(value=-100, inplace=True)
X['atl_change_percentage'].fillna(value=-100, inplace=True)
X['total_supply'].fillna(value=-100, inplace=True)
X['price_change_percentage_60d_in_currency'].fillna(value=-100, inplace=True)

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12703 entries, 0 to 12702
Data columns (total 31 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   id                                       12703 non-null  object        
 1   symbol                                   12703 non-null  object        
 2   name                                     12703 non-null  object        
 3   asset_platform_id                        12703 non-null  object        
 4   block_time_in_minutes                    12703 non-null  object        
 5   hashing_algorithm                        12703 non-null  object        
 6   categories                               12703 non-null  object        
 7   description                              12703 non-null  object        
 8   country_origin                           12703 non-null  object        
 9   genesis_date                           

In [12]:
dictnames = ['description','market_data','community_data','developer_data','public_interest_stats']

numvars = ['sentiment_votes_up_percentage',
'sentiment_votes_down_percentage',
'market_cap_rank', 'mcap_to_tvl_ratio',
'coingecko_rank','fdv_to_tvl_ratio',
'coingecko_score','market_cap',
'developer_score','price_change_percentage_60d_in_currency',
'community_score','total_supply',
'liquidity_score','ath_change_percentage',
'public_interest_score','atl_change_percentage',
'block_time_in_minutes','Jdate']

strvars = ['asset_platform_id',
'hashing_algorithm',
'country_origin',
]

# 'categories' need to go through dictencoder

In [13]:
# turns strings in a list into dictionary with string as key and 1, 0 as values
class DictEncoder(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # X will be a pandas series. Return a pandas series of dictionaries
        self = pd.Series([], dtype= object)
              
        for rowi in X.index:
            row = X[rowi]
            self[rowi] = dict(zip(row, [1]*len(row)))

        return self
    
    
class DictFlat(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # X is data converted into a dataframe
        X_out = pd.Series([], dtype='O')
        
        for rowi in X.index:
            
            tempdic = X.loc[rowi]
            attrs = pd.json_normalize(tempdic, sep='_')
            attrs_flat = pd.DataFrame.to_dict(attrs, orient='index')
            #mydict = {'en':attrs_flat[0]['en']}
            X_out[rowi] = {'en':attrs_flat[0]['en']}
            
        return X_out

In [14]:
# should be performed on 'categories'
list2Dict = DictEncoder()
Tfidf = TfidfVectorizer()
dictFlatten = DictFlat()
scaler = StandardScaler()
DV = DictVectorizer()

In [15]:
strPipe = Pipeline([('Tfidf_Vectorizer', Tfidf)])

catPipe = Pipeline([('List_to_Dict', list2Dict), ('Dict2Vec', DictVectorizer())])

dictPipe = Pipeline([('DictFlat', dictFlatten), 
                    ('Dict2Vec', DV)])


In [16]:
preprocessor = ColumnTransformer(
        transformers=[
        ("platform", TfidfVectorizer(), 'asset_platform_id'), ("hash-alg", TfidfVectorizer(), 'hashing_algorithm'),
        ("country", TfidfVectorizer(), 'country_origin'),("cat", catPipe, 'categories'), ("numerical", scaler, numvars)],
                             remainder='drop')

X_out = preprocessor.fit_transform(X)

In [17]:
X_out.shape

(12703, 403)

In [32]:
db = DBSCAN(eps=1.6, min_samples=20).fit(X_out)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [33]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X_out, labels))

Estimated number of clusters: 5
Estimated number of noise points: 3724
Silhouette Coefficient: 0.067


In [34]:
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(X_out)

In [35]:
temp = np.hstack((X_embedded, labels.reshape((-1,1))))
dfplot = pd.DataFrame(data=temp, index=X.loc[:,'name'], columns=["X", "Y", "label"]).reset_index()

In [37]:
fig = px.scatter(dfplot, x="X", y="Y", color="label", height=600, width=1000, hover_name="name", title="Clusters of all Coins")
fig.update_coloraxes(showscale=False)
fig.show()  

In [24]:
cg = CoinGeckoAPI()
trending = cg.get_search_trending()

In [25]:
mycoins = [coin['item']["id"] for coin in trending['coins']]
mycoins

['apecoin', 'dehub', 'railgun', 'jasmycoin', 'avalanche-2', 'stepn', 'fantom']

In [26]:
for coin in mycoins:
    current_coin = cg.get_coin_market_chart_by_id(coin, 'usd', 30)
    df0 = pd.DataFrame.from_dict(current_coin)
    df0['dt'] = df0[['prices']].applymap(lambda x: x[0])
    df0['dt'] = df0[['dt']].applymap(lambda x: round(x/1000))
    df0['dt'] = df0[['dt']].applymap(lambda x: datetime.fromtimestamp(x))
    df0['dt'] = df0['dt'].dt.ceil(freq='H')
    df0[['prices','market_caps','total_volumes']] = df0[['prices','market_caps','total_volumes']].applymap(lambda x: x[1])
    if coin == mycoins[0]:
        temp = df0[['dt','prices']]
        df2plot = temp.rename(columns={'dt':'dt', 'prices':coin})
        print(df2plot.shape)
    else:
        temp = df0[['dt','prices']]
        temp = temp.rename(columns={'dt':'dt', 'prices':coin})
        df2plot = df2plot.merge(temp, on='dt', how='outer')
        print(temp.shape)

(37, 2)
(721, 2)
(721, 2)
(721, 2)
(725, 2)
(229, 2)
(725, 2)


In [27]:
df2plot = df2plot.set_index('dt')
df2plot.rename_axis("coinid", axis=1, inplace=True)

In [38]:
fig = px.area(df2plot, facet_col="coinid", facet_col_wrap=2, title="Prices for Trending Coins")
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("coinid=", "")))
fig.update_layout(showlegend=False)
fig.show()